In [11]:
import pytesseract
from PIL import Image
import pdf2image
import re

In [25]:
pdf_file = "fetched_data/result/56.1463551be%201-2.pdf"


In [13]:
pattern = r"2?\d+"
matches = re.findall(pattern,pdf_file)

In [14]:
matches = matches[-1] if matches[0][0] == "2" else "2"+matches[-1]
print(matches)

22


In [17]:
images = pdf2image.convert_from_path(pdf_file)

In [18]:
languages = ['eng', 'nep']

# Set the page segmentation mode to use automatic page segmentation with OSD
psm = '--psm 6'

# Set the OCR engine mode to use both LSTM and Legacy OCR engines
oem = '--oem 3'

# Build the configuration string
config = f'{psm} {oem} -l {"+".join(languages)}'


In [19]:
faculty_level_pattern = r'(I|II|III|IV)/\s*(I|II|III|IV)\s+'
text1 = pytesseract.image_to_string(images[0])
text1 = text1.split("Page")[0].replace("l","I")
faculty = re.findall(faculty_level_pattern,text1)
print(faculty)


[('I', 'II')]


In [20]:
text_append = []
for pg_no,image in enumerate(images):
    if pg_no == 0:
        text = pytesseract.image_to_string(image,config=config)
        text = text.split("।")[-1].strip()
    elif pg_no < len(images)-1:
        text = pytesseract.image_to_string(image,config="--psm 6")
        text = text.split("Nepal")[-1].strip()
    elif pg_no == len(images)-1:
        text = pytesseract.image_to_string(image,config=config)
        text = text.split("Nepal")[-1].strip()
        text = "".join(text.split("परीक्षा")[0]).strip()

    text_append.append(text.strip().replace(" &",",").replace(".",""))


In [21]:
input_txt = "".join(text_append)

In [22]:
# Define the regular expression pattern to match the faculty names and their symbol numbers
pattern = r'(BCT|BCE|BEX|BME|BAM|BGE|BAE|BEL)\s+(\w+/?\w*)\s+((?:\d+(?:,\s*)?)+)'
# Iterate over the matches in the input text and create a dictionary of faculty names and symbol numbers
symbol_dict = {}
for match in re.finditer(pattern, input_txt):
    faculty_name = match.group(1)
    faculty_level = match.group(2)
    # symbol_numbers = [int(x) for x in match.group(3).split(',')]
    symbol_numbers = re.findall(r'\b\d+\b',match.group(3))
    key = faculty_name
    symbol_dict[key] = symbol_numbers


In [43]:
print(symbol_dict["BCT"])

['3654', '3655', '3659', '3661', '3662', '3663', '3665', '3666', '3668', '3702', '3705', '3710', '3712', '3713', '3714', '3724', '3725', '3731', '3733', '3734', '3736', '3738', '3740', '3741', '3743', '3744', '3745', '3751', '3852', '3854', '3856', '3858', '3860', '3861', '3862', '3867', '3868', '3901', '3902', '3903', '3904', '3905', '3958', '3959', '3963', '3964', '4002', '4004', '4006', '4052', '4053', '4104', '4106', '4109', '4111', '4118', '4151', '4152', '4201', '4259', '4260', '4261', '4262', '4263', '4264', '4265', '4301', '4309', '4310', '4315', '4320', '4321', '4323', '4324', '4325', '4329']


In [23]:
print(symbol_dict.keys())

dict_keys(['BCE', 'BEL', 'BEX', 'BCT', 'BME', 'BAM', 'BGE'])
